In [1]:
import requests
import base64

In [2]:
# Replace with your own Client ID and Client Secret
CLIENT_ID = 'a3067336aca542788fc90ec43eba9ced'
CLIENT_SECRET = '913d0de4588b4388a5cb26704185587e'

In [3]:
# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [4]:
# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}

In [5]:
response = requests.post(token_url, data=data, headers=headers)

In [6]:
if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [7]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import warnings
warnings.filterwarnings('ignore')

/tmp/ipykernel_108428/607799181.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [9]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

In [10]:
# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,redrum,21 Savage,american dream,2RRYaYHY7fIIdvFlvgb5vq,52eIcoLUM25zbQupAZYoFh,93,2024-01-12,270698,True,https://open.spotify.com/track/52eIcoLUM25zbQu...,...,0.740,2,-8.445,1,0.0481,0.00529,0.000224,0.5000,0.246,172.089
1,Lovin On Me,Jack Harlow,Lovin On Me,6VCO0fDBGbRW8mCEvV95af,4xhsWYTOGcal8zt0J161CU,93,2023-11-10,138411,True,https://open.spotify.com/track/4xhsWYTOGcal8zt...,...,0.558,2,-4.911,1,0.0568,0.00260,0.000002,0.0937,0.606,104.983
2,BELLAKEO,"Peso Pluma, Anitta",BELLAKEO,3VLY9g3CAG1Y5r2eGVEaZ0,05WVKTdZhlIMX4qqMLuo0f,89,2023-12-07,197333,True,https://open.spotify.com/track/05WVKTdZhlIMX4q...,...,0.880,9,-2.834,1,0.1010,0.05620,0.060000,0.1530,0.463,180.011
3,I'm Good (Blue),"David Guetta, Bebe Rexha",I'm Good (Blue),7M842DMhYVALrXsw3ty7B3,4uUG5RXrOk84mYEfFvj3cK,89,2022-08-26,175238,True,https://open.spotify.com/track/4uUG5RXrOk84mYE...,...,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040
4,Prada,"cassö, RAYE, D-Block Europe",Prada,5MU0RmBSpoSxOPYBfcobDc,59NraMJsLaMCVtwXTSia8i,89,2023-08-11,132359,True,https://open.spotify.com/track/59NraMJsLaMCVtw...,...,0.717,8,-5.804,1,0.0375,0.00100,0.000002,0.1130,0.422,141.904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,La Llevo Al Cielo (Ft. Ñengo Flow),"Chris Jedi, Anuel AA, Chencho Corleone, Ñengo ...",La Llevo Al Cielo (Ft. Ñengo Flow),0WEtvlRZhn9bZLHbjCLSv2,6DoL1yYIwEW7VZMRaJhoJI,74,2022-05-20,254920,True,https://open.spotify.com/track/6DoL1yYIwEW7VZM...,...,0.845,9,-3.809,0,0.1350,0.10500,0.000011,0.1720,0.769,170.023
96,Another Love - Tiësto Remix,"Tom Odell, Tiësto",Another Love (Tiësto Remix),1QltVGlNGbK94CKgUsYZGa,4Et6tUEO7biKxZEfJXpNj1,74,2022-05-13,185366,False,https://open.spotify.com/track/4Et6tUEO7biKxZE...,...,0.764,4,-4.685,0,0.0737,0.08360,0.087100,0.1320,0.473,123.046
97,WORTH NOTHING (feat. Oliver Tree) - Fast & Fur...,"TWISTED, Oliver Tree, Fast & Furious: The Fast...",WORTH NOTHING (feat. Oliver Tree) [Fast & Furi...,1nyfZlIOe109tZTrpR3z8h,3PjbA0O5olhampPMdaB0V1,74,2022-09-28,164629,True,https://open.spotify.com/track/3PjbA0O5olhampP...,...,0.601,2,-7.887,0,0.0628,0.15000,0.000579,0.1680,0.156,139.880
98,Remember,"Becky Hill, David Guetta",Remember,6DHfD3rZapWFmEKX1FNNZF,4laAKIq9ZxBCwf99rauPYb,74,2021-06-18,160766,False,https://open.spotify.com/track/4laAKIq9ZxBCwf9...,...,0.862,8,-2.903,1,0.0370,0.04100,0.000000,0.0907,0.354,123.849


In [11]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [13]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [14]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [15]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [16]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec

    temp_df = pd.DataFrame(data={
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, index=[])
    # print(temp_df)
    
    hybrid_recommendations = pd.concat([hybrid_recommendations, temp_df], ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [17]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

Hybrid recommended songs for 'I'm Good (Blue)':


,Track Name,Artists,Album Name,Release Date,Popularity
2,Call It Love,"Felix Jaehn, Ray Dalton",Call It Love,2022-09-16,79.0
0,REACT,"Switch Disco, Ella Henderson, Robert Miles",REACT,2023-01-13,76.0
1,Where You Are,"John Summit, Hayla",Where You Are,2023-03-03,76.0
4,Where Did You Go? (feat. MNEK),"Jax Jones, MNEK",Where Did You Go (feat. MNEK),2022-02-04,76.0
3,Rainfall (Praise You),Tom Santa,Rainfall (Praise You),2022-02-18,75.0
